In [30]:
import pandas as pd
import plotly.express as px
import re

In [31]:
# Define the results you want to explore
csv_path = "./results_cellpdia30_sigma6_dilrad4_dnad_obj_seg_v1_gliaero5_gliathr20.csv" 

# Read the CSV file
df = pd.read_csv(csv_path)

# Convert the index into a column
df.reset_index(inplace=True)

# Extract 'tissue_location'
df['tissue_location'] = df['filename'].str.split('_40X_').str[-1]

# Extract 'staining_id'
df['staining_id'] = df['filename'].str.extract('(\d+)_40X')

# Display the first few rows of the DataFrame
df.head()


,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,%_dna_damage_signal,%_glia+_signal,tissue_location,staining_id
0,0,DSB Iba1 16_40X_CA1,0.892857,1.898734,0.888889,1.814433,79,89,168,198,1.096630,88.633728,CA1,16
1,1,DSB Iba1 16_40X_CA3,4.056497,4.273810,4.311475,4.508571,168,9,177,183,18.227673,96.349525,CA3,16
2,2,DSB Iba1 16_40X_CTX1,1.346939,2.444444,0.732510,1.873684,27,22,49,243,0.844288,13.598442,CTX1,16
3,3,DSB Iba1 16_40X_CTX2,2.420833,2.472340,2.698795,2.754098,235,5,240,249,39.405441,94.930744,CTX2,16
4,4,DSB Iba1 16_40X_CTX3,2.188406,2.359375,2.035874,2.304569,128,10,138,223,9.070396,54.768944,CTX3,16


In [32]:
# Extract analysis parameters from the CSV path
extracted_values = re.findall(r'\d+', csv_path)

# Dynamically assign the extracted values to variables
if len(extracted_values) >= 6:
    cellpose_nuclei_diameter = int(extracted_values[0])
    gaussian_sigma = int(extracted_values[1])
    dilation_radius_nuclei = int(extracted_values[2])
    dna_damage_segmenter_version = int(extracted_values[3])
    glia_nuclei_colocalization_erosion = int(extracted_values[4])
    if "_glia_sem_seg_v" in str(csv_path):
        glia_segmenter = True
    else:
        glia_segmenter = False

if glia_segmenter:
    glia_segmenter_version = int(extracted_values[5])
    glia_channel_threshold = None
    # Dinamically adjust plot titles
    title = f"cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_glia_sem_seg_v{glia_segmenter_version}"
else:
    glia_channel_threshold = int(extracted_values[5])
    glia_segmenter_version = None
    # Dinamically adjust plot titles
    title = f"cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_gliathr{glia_channel_threshold}"
    

# Print the assigned analysis parameters
print(f"Cellpose nuclei diameter: {cellpose_nuclei_diameter}")
print(f"Gaussian sigma: {gaussian_sigma}")
print(f"Dilation radius nuclei: {dilation_radius_nuclei}")
print(f"Dna damage segmenter version: {dna_damage_segmenter_version}")
print(f"Glia erosion: {glia_nuclei_colocalization_erosion}")
print(f"Glia threshold: {glia_channel_threshold}")
print(f"Glia semantic segmentation version: {glia_segmenter_version}")



Cellpose nuclei diameter: 30
Gaussian sigma: 6
Dilation radius nuclei: 4
Dna damage segmenter version: 1
Glia erosion: 5
Glia threshold: 20
Glia semantic segmentation version: None


In [33]:
mouse_id_csv_path = "./mouse_ids.csv"

# Read the CSV file
df_mouse_id = pd.read_csv(mouse_id_csv_path, delimiter=";", encoding="UTF-8")

# Display the first few rows of the DataFrame
df_mouse_id.head()

,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc
0,34,885,male,APP/PS1,poor,passed
1,53,885,male,APP/PS1,good,passed
2,23,2042,male,APP/PS1,good,passed
3,31,2042,male,APP/PS1,good,passed
4,43,2042,male,APP/PS1,good,passed


In [34]:
df['staining_id'] = pd.to_numeric(df['staining_id'], errors='coerce')
df_mouse_id['staining_id'] = pd.to_numeric(df_mouse_id['staining_id'], errors='coerce')

# Merge both processed_results_df and mouse_id dataframes on staining_id
merged_df = pd.merge(df, df_mouse_id, on="staining_id")

# Display the first few rows of the DataFrame
merged_df.head()

,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,%_dna_damage_signal,%_glia+_signal,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc
0,0,DSB Iba1 16_40X_CA1,0.892857,1.898734,0.888889,1.814433,79,89,168,198,1.096630,88.633728,CA1,16,887,male,APP/PS1,poor,failed
1,1,DSB Iba1 16_40X_CA3,4.056497,4.273810,4.311475,4.508571,168,9,177,183,18.227673,96.349525,CA3,16,887,male,APP/PS1,poor,failed
2,2,DSB Iba1 16_40X_CTX1,1.346939,2.444444,0.732510,1.873684,27,22,49,243,0.844288,13.598442,CTX1,16,887,male,APP/PS1,poor,failed
3,3,DSB Iba1 16_40X_CTX2,2.420833,2.472340,2.698795,2.754098,235,5,240,249,39.405441,94.930744,CTX2,16,887,male,APP/PS1,poor,failed
4,4,DSB Iba1 16_40X_CTX3,2.188406,2.359375,2.035874,2.304569,128,10,138,223,9.070396,54.768944,CTX3,16,887,male,APP/PS1,poor,failed


In [35]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_+_dna_damage_glia_nuclei',
                 hover_data=['staining_id','index'], title=f"Number of DNA damage+ Glia+ Nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [36]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='avg_dna_damage_foci/glia_+',
                 hover_data=['staining_id'], title=f"Average DNA damage foci in Glia Nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [37]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei',
                 hover_data=['staining_id'], title=f"Average DNA damage foci in All Nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [38]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_glia_+_nuclei',
                 hover_data=['staining_id','index'], title=f"Nr of glia+ nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [39]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_total_nuclei',
                 hover_data=['staining_id','index'], title=f'Nr of total nuclei by Tissue Location - {title}')

# Show the plot
fig.show()

In [40]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='nr_glia_+_nuclei',
                 hover_data=['tissue_location','index'], title=f'Nr of glia+ nuclei by Sample - {title}')

# Show the plot
fig.show()

In [41]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='%_dna_damage_signal',
                 hover_data=['tissue_location','index'], title=f'Dna damage mask area (QC) - {title}')

# Show the plot
fig.show()

In [42]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='%_glia+_signal',
                 hover_data=['tissue_location','index'], title=f'Glia mask area (QC) - {title}')

# Show the plot
fig.show()

We can observe there is a number of outliers in the glial and dna damage mask detection given the staining is suboptimal in some of the samples. I will filter the data to remove those suboptimal stains and just plot the optimal ones where the automated image analysis offers reliable results.

In [43]:
# Calculate mean area of the image occupied by glia+ signal
glia_mask_area_mean = df['%_glia+_signal'].mean() 

# Calculate mean area of the image occupied by dna_damage_+ signal
dna_damage_mask_area_mean = df['%_dna_damage_signal'].mean() 

# Print extracted values
print(f"Glia_mask_area_%_mean: {glia_mask_area_mean}, Dna_damage_mask_area_%_mean: {dna_damage_mask_area_mean}") 

Glia_mask_area_%_mean: 11.002078888908265, Dna_damage_mask_area_%_mean: 10.337142717270623


In [44]:
# Define a function to determine staining quality, anything above 3 times the mean value is considered an outlier
def determine_stain_quality(value, mean_value):
    if value < (mean_value + mean_value*3):
        return "optimal"
    else:
        return "suboptimal"

# Check stain quality for glia and create another column storing optimal or suboptimal if qc_passed or not    
merged_df['glia_stain_quality_auto'] = merged_df['%_glia+_signal'].apply(lambda x: determine_stain_quality(x, glia_mask_area_mean))

# Check stain quality for dna_damage and create another column storing optimal or suboptimal if qc_passed or not 
merged_df['dna_damage_stain_quality_auto'] = merged_df['%_dna_damage_signal'].apply(lambda x: determine_stain_quality(x, dna_damage_mask_area_mean))

# Check for both stain qualities and store True qc_passed if both are optimal
merged_df['staining_qc_passed'] = (merged_df['glia_stain_quality_auto'] == 'optimal') & (merged_df['dna_damage_stain_quality_auto'] == 'optimal')

# Group the DataFrame by 'staining_id' and check if all 'staining_qc_passed' values are True, otherwise set them all to False
merged_df['staining_qc_passed'] = merged_df.groupby('staining_id')['staining_qc_passed'].transform('all')

# Now, if all 'staining_qc_passed' values for the same 'staining_id' were True, the column will remain True; otherwise, it will be False

merged_df.head()

,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,...,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc,glia_stain_quality_auto,dna_damage_stain_quality_auto,staining_qc_passed
0,0,DSB Iba1 16_40X_CA1,0.892857,1.898734,0.888889,1.814433,79,89,168,198,...,CA1,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
1,1,DSB Iba1 16_40X_CA3,4.056497,4.273810,4.311475,4.508571,168,9,177,183,...,CA3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
2,2,DSB Iba1 16_40X_CTX1,1.346939,2.444444,0.732510,1.873684,27,22,49,243,...,CTX1,16,887,male,APP/PS1,poor,failed,optimal,optimal,False
3,3,DSB Iba1 16_40X_CTX2,2.420833,2.472340,2.698795,2.754098,235,5,240,249,...,CTX2,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
4,4,DSB Iba1 16_40X_CTX3,2.188406,2.359375,2.035874,2.304569,128,10,138,223,...,CTX3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False


In [45]:
if glia_segmenter:
    merged_df.to_csv(
        f"qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_glia_sem_seg_v{glia_segmenter_version}.csv",
        index=False,
    )
else:
    merged_df.to_csv(
        f"qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_gliathr{glia_channel_threshold}.csv",
        index=False,
    )

In [46]:
# Remove data from images with a poor quality stain (auto QC)
auto_filtered_df = merged_df[merged_df['staining_qc_passed'] == True]

In [47]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/glia_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Glia Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

c:\Users\albertds\mambaforge\envs\microglia\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [48]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/glia_+_damage_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Damaged Glia Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

c:\Users\albertds\mambaforge\envs\microglia\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [49]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in All Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

c:\Users\albertds\mambaforge\envs\microglia\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [50]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei_damage_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Damage+ Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

c:\Users\albertds\mambaforge\envs\microglia\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Show failed qc dataframe

In [51]:
qc_failed_df = merged_df[merged_df['staining_qc_passed'] == False]

print(f"{qc_failed_df.shape[0]} stains have not passed QC and have been discarded")

qc_failed_list = qc_failed_df['index'].tolist()

qc_failed_df


58 stains have not passed QC and have been discarded


,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,...,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc,glia_stain_quality_auto,dna_damage_stain_quality_auto,staining_qc_passed
0,0,DSB Iba1 16_40X_CA1,0.892857,1.898734,0.888889,1.814433,79,89,168,198,...,CA1,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
1,1,DSB Iba1 16_40X_CA3,4.056497,4.273810,4.311475,4.508571,168,9,177,183,...,CA3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
2,2,DSB Iba1 16_40X_CTX1,1.346939,2.444444,0.732510,1.873684,27,22,49,243,...,CTX1,16,887,male,APP/PS1,poor,failed,optimal,optimal,False
3,3,DSB Iba1 16_40X_CTX2,2.420833,2.472340,2.698795,2.754098,235,5,240,249,...,CTX2,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
4,4,DSB Iba1 16_40X_CTX3,2.188406,2.359375,2.035874,2.304569,128,10,138,223,...,CTX3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
5,5,DSB Iba1 16_40X_DG,0.990338,1.798246,0.990783,1.791667,114,93,207,217,...,DG,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
6,6,DSB Iba1 17_40X_CA1,1.909091,3.000000,2.832258,3.850877,7,4,11,155,...,CA1,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
7,7,DSB Iba1 17_40X_CA3,1.750000,2.625000,3.842697,4.750000,16,8,24,178,...,CA3,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
8,8,DSB Iba1 17_40X_CTX1,2.428571,2.833333,1.384298,2.233333,18,3,21,242,...,CTX1,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
9,9,DSB Iba1 17_40X_CTX2,3.568966,3.696429,3.068592,3.195489,56,2,58,277,...,CTX2,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
